# Showing HPO Annotations

## Setup

In [1]:
## CX: allows multiple lines of code to print from one code block
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## to get around bugs
import nest_asyncio
nest_asyncio.apply()

import pathlib
import pandas as pd
import re
import requests

from collections import defaultdict

In [2]:
## put the path you want to use here
HPO_path = pathlib.Path.cwd().joinpath('rawData', 'phenotype.hpoa')

In [3]:
## from https://github.com/biothings/mydisease.info/blob/master/src/plugins/hpo/parser.py

def process_disease2hp(file_path_disease_hpo):
    df_disease_hpo = pd.read_csv(file_path_disease_hpo, sep="\t", skiprows=4, dtype=str)
    df_disease_hpo = df_disease_hpo.rename(
        index=str, columns={"DiseaseName": "disease_name", "#DatabaseID": "disease_id"}
    )
    
    ## removing qualifier = 'NOT' annotations, because it means the disease does not
    ##   have this phenotypic feature. The HPO website doesn't show these 'NOT' annots
    df_disease_hpo = df_disease_hpo[df_disease_hpo['Qualifier'] != "NOT"]
    ## then remove the qualifier
    df_disease_hpo.drop(columns = 'Qualifier', inplace = True)
    ## make sure all null values are None
    df_disease_hpo = df_disease_hpo.where((pd.notnull(df_disease_hpo)), None)
    
    d = []
    for did, subdf in df_disease_hpo.groupby("disease_id"):
        did = did.replace("ORPHA", "ORPHANET")
        records = subdf.to_dict(orient="records")
        
        pathway_related = []
        course = []
        modifiers = []
        inheritance = []
        
        for record in records:
            record_dict = {}
            if record["Aspect"] == "C":
                course.append(record["HPO_ID"])
                continue
            elif record["Aspect"] == "M":
                modifiers.append(record["HPO_ID"])
                continue
            elif record["Aspect"] == "I":
                inheritance.append(record["HPO_ID"])
                continue      
                
            for k, v in record.items():
                # name the field based on pathway database
                if (k == "Sex") and v:
                    record_dict['sex'] = v.lower()
                elif (k == 'Reference') and v: 
                ## only process if Reference has a value
                ## notes: OMIM:194190, OMIM:180849, OMIM:212050 are disease examples with > 1 type of reference
                    ## this is a string representing a list
                    tempRefs = v.split(";")
                    ## prepare to iterate through the tempRefs and store the processed data
                    tempProperties = {
                        'ISBN': [],
                        'PMID': [],
                        'http': [],
                        'DECIPHER': [],
                        'OMIM': [],
                        'ORPHA': []
                    }
                    
                    ## remove the prefixes or not? currently keeping the prefix
                    for i in tempRefs:
                        for key in tempProperties.keys():
                            if key in i:
                                ## replace curie prefix for isbn and orpha
                                if key == 'ISBN':
                                    tempProperties[key].append('ISBN:' + i.split(":")[1])
                                elif key == 'ORPHA':
                                    tempProperties[key].append('ORPHANET:' + i.split(":")[1])                                    
                                else:
                                    tempProperties[key].append(i)
                    ## ONLY add reference keys/values to the record if there are values
                    for k,v in tempProperties.items():
                        if v:
                            if k == 'ISBN':
                                record_dict['isbn_refs'] = v
                            elif k == 'PMID':
                                record_dict['pmid_refs'] = v                    
                            elif k == 'http':
                                record_dict['website_refs'] = v  
                            elif k == 'DECIPHER':
                                record_dict['decipher_refs'] = v  
                            elif k == 'OMIM':
                                record_dict['omim_refs'] = v  
                            elif k == 'ORPHA':
                                record_dict['orphanet_refs'] = v  
                elif (k == 'Frequency') and v:
                ## only process if Frequency has a value
                    tempDict = {}
                    if 'http' in v:  ## catching an error in the data
                        continue
                    elif 'HP:' in v:
                        tempDict['hp_freq'] = v
                    elif '%' in v:
                        tempFreq = float(v.strip('%')) / 100
                        ## only go forward if this is a valid fraction <=1
                        if tempFreq <= 1:
                            tempDict['numeric_freq'] = tempFreq
                    elif '/' in v:
                        ## idx 0 is numerator, idx 1 is denominator
                        tempL = [int(ele) for ele in v.split("/")]
                        ## only go forward if this is a valid fraction <=1
                        if (tempL[0] != 0) and (tempL[1] !=0) and (tempL[0] <= tempL[1]):
                            tempDict['freq_numerator'] = tempL[0]
                            tempDict['freq_denominator'] = tempL[1]
                            tempDict['numeric_freq'] = tempL[0] / tempL[1]

                    ## ONLY add frequency keys/values to the record if there are values
                    if tempDict:
                        record_dict.update(tempDict)
                        
                elif (k == 'Modifier') and v:
                ## only process if Modifier has a value
                    ## in <20 records, this is a delimited list with repeated values
                    ## this behavior matches the unlist behavior used with biothings APIs
                    ## https://github.com/kevinxin90/biothings.api/blob/master/biothings/utils/dataload.py
                    if ";" in v:
                        ## transform to list -> set->list to remove repeated values
                        tempMods = list(set(v.split(";")))
                        record_dict['modifier'] = tempMods
                    else:
                        record_dict['modifier'] = v
                elif k not in {"disease_id", "disease_name", 
                               "Aspect", "Sex", 
                               "Reference", "Frequency", 
                               "Modifier"}:
                    record_dict[k.lower()] = v
            pathway_related.append(record_dict)
        drecord = {
            "_id": did,
            "hpo": pathway_related,
            "disease_name": records[0]["disease_name"],
            "course": course,
            "modifiers": modifiers,
            "inheritance": inheritance
        }
        d.append(drecord)
 
    return {
        x["_id"]: [x["hpo"], x["disease_name"], x["course"], x["modifiers"], x["inheritance"]] for x in d
    }

In [4]:
processed_data = process_disease2hp(HPO_path)

## Comparing records

Temtamy syndrome has [an OMIM ID (218340) and Orphanet ID (1777)](https://www.orpha.net/consor/cgi-bin/OC_Exp.php?lng=EN&Expert=1777). These are mapped to the same MONDO ID in mydisease.info (this will be shown later below)

In [5]:
getMyDiseaseMapping = requests.get("http://mydisease.info/v1/query?q=%22MONDO%3A0009033%22&fields=mondo.xrefs")
getMyDiseaseMapping = getMyDiseaseMapping.json()

getMyDiseaseMapping['hits'][0]

{'_id': 'MONDO:0009033',
 '_score': 5.4348025,
 'mondo': {'xrefs': {'doid': ['DOID:0111621'],
   'gard': ['5688'],
   'icd10': ['Q87.8'],
   'mesh': ['C536959'],
   'ncit': ['C148371'],
   'omim': ['218340'],
   'orphanet': ['1777'],
   'sctid': ['719947004'],
   'umls': ['C1857512']}}}

These are separate records in the HPO annotations, with different phenotype data annotated to each one

Compare the info below:

In [6]:
OMIMrecord = processed_data['OMIM:218340']
Orphanetrecord = processed_data['ORPHANET:1777']

In [7]:
print("OMIM has {} phenotypes".format(len(OMIMrecord[0])))
print("Orphanet has {} phenotypes".format(len(Orphanetrecord[0])))
print("\n")

print("OMIM disease name: {}".format(OMIMrecord[1]))
print("Orphanet disease name: {}".format(Orphanetrecord[1]))
print("\n")

print("OMIM course info: {}".format(OMIMrecord[2]))
print("Orphanet course info: {}".format(Orphanetrecord[2]))
print("\n")

print("OMIM modifier info: {}".format(OMIMrecord[3]))
print("Orphanet modifier info: {}".format(Orphanetrecord[3]))
print("\n")

print("OMIM inheritance info: {}".format(OMIMrecord[4]))
print("Orphanet inheritance info: {}".format(Orphanetrecord[4]))

OMIM has 31 phenotypes
Orphanet has 25 phenotypes


OMIM disease name: CRANIOFACIAL DYSMORPHISM WITH OCULAR COLOBOMA, ABSENT CORPUS CALLOSUM,AND AORTIC DILATATION
Orphanet disease name: Temtamy syndrome


OMIM course info: ['HP:0003593']
Orphanet course info: []


OMIM modifier info: []
Orphanet modifier info: []


OMIM inheritance info: ['HP:0000007']
Orphanet inheritance info: []


These two records are very different! You can also see this on the webpages for the [OMIM](https://hpo.jax.org/app/browse/disease/OMIM:218340) and [Orphanet](https://hpo.jax.org/app/browse/disease/ORPHA:1777) annotations.

In [8]:
setOMIMphenotypes = set([pheno['hpo_id'] for pheno in OMIMrecord[0]])
setOrphaphenotypes = set([pheno['hpo_id'] for pheno in Orphanetrecord[0]])

### HPO phenotypes in both records

In [9]:
len(setOrphaphenotypes.intersection(setOMIMphenotypes))

11

### HPO phenotypes in the OMIM record, but not in the Orphanet record

In [10]:
len(setOMIMphenotypes - setOrphaphenotypes)

20

### HPO phenotypes in the Orphanet record, but not in the OMIM record

In [11]:
len(setOrphaphenotypes - setOMIMphenotypes)

14

### Compared to what's in MyDisease.info

Compare this with what mydisease.info has for Temtamy syndrome [MONDO:0009033](https://monarchinitiative.org/disease/MONDO:0009033). It seems to have the data from the OMIM ID but not the Orphanet ID...

In [12]:
getMyDisease = requests.get('http://mydisease.info/v1/query?q="MONDO:0009033"&fields=hpo')
getMyDisease = getMyDisease.json()['hits'][0]['hpo']

OMIM phenotypes vs MyDisease phenotypes:
- the difference is likely due to the mydisease data being from an older HPOA release
- 1 phenotype is in the latest (April 2021) OMIM annotations, but not in the MyDisease.info 
- all of the phenos in mydisease are in the omim annots

In [13]:
setMyDiseasephenos = set([pheno['hpo_id'] for pheno in getMyDisease['phenotype_related_to_disease']])

setOMIMphenotypes - setMyDiseasephenos

setMyDiseasephenos - setOMIMphenotypes

{'HP:0001252'}

set()

Orphanet phenotypes vs MyDisease phenotypes: 

- 14 phenotypes are in the orphanet annots but not the mydisease entry - matches the "in orphanet, not in omim" count above  
- 19 phenotypes are in the mydisease entry but not the latest (April 2021) orphanet annots - if you add the "new" omim annot above, then the number would be 20 which matches the "in omim, not in orphanet" count above.

In [14]:
len(setOrphaphenotypes - setMyDiseasephenos)

len(setMyDiseasephenos - setOrphaphenotypes)

14

19

In [15]:
print("myDisease finds {} phenotypes".format(len(getMyDisease['phenotype_related_to_disease'])))
print("\n")

print("in myDisease, the disease name from HPO is {}".format(getMyDisease['disease_name']))
print("\n")

print("in myDisease, the course is {}".format(getMyDisease['course']))
print("\n")

print("in myDisease, the inheritance is {}".format(getMyDisease['inheritance']))

myDisease finds 30 phenotypes


in myDisease, the disease name from HPO is CRANIOFACIAL DYSMORPHISM WITH OCULAR COLOBOMA, ABSENT CORPUS CALLOSUM,AND AORTIC DILATATION


in myDisease, the course is HP:0003593


in myDisease, the inheritance is HP:0000007


## Why is this happening?

The parser uses an [if-elif-else structure](https://github.com/biothings/mydisease.info/blob/f59108a8e83cd53f18bcd2e542fb922a9d88137f/src/plugins/hpo/parser.py#L174) to put only 1 record that maps to the MONDO ID into the final data for the API. This means that a disease like Temtamy syndrome, which maps to an OMIM and Orphanet ID, only has the OMIM ID's data in the final API

To fix this, there may need to be discussion on how to merge the records...

Notice how the reference, evidence, frequency, and biocuration data differs between annotation for the same phenotype in the different records:

In [16]:
for pheno in OMIMrecord[0]:
    if pheno['hpo_id'] == 'HP:0000276':
        pheno

        
for pheno in Orphanetrecord[0]:
    if pheno['hpo_id'] == 'HP:0000276':
        pheno

{'hpo_id': 'HP:0000276',
 'omim_refs': ['OMIM:218340'],
 'evidence': 'IEA',
 'onset': None,
 'biocuration': 'HPO:iea[2009-02-17]'}

{'hpo_id': 'HP:0000276',
 'orphanet_refs': ['ORPHANET:1777'],
 'evidence': 'TAS',
 'onset': None,
 'hp_freq': 'HP:0040282',
 'biocuration': 'ORPHA:orphadata[2021-04-14]'}

The set of shared phenotypes, in case one wants to look at the records and how they differ from each other

In [17]:
setOrphaphenotypes.intersection(setOMIMphenotypes)

{'HP:0000276',
 'HP:0000316',
 'HP:0000347',
 'HP:0000369',
 'HP:0000444',
 'HP:0000567',
 'HP:0000612',
 'HP:0001156',
 'HP:0001263',
 'HP:0001763',
 'HP:0004942'}